In [2]:
import tweepy
import pandas as pd
from pyspark.sql import functions as F
import numpy as np
import json
from dotenv import load_dotenv
import os
import datetime

In [3]:
load_dotenv()

# Inicializar o cliente do Tweepy
client = tweepy.Client(consumer_key=os.getenv('TWEEPY_CONSUMER_KEY')
                     , consumer_secret=os.getenv('TWEEPY_CONSUMER_SECRET')
                     , bearer_token=os.getenv('TWEEPY_BEARER_TOKEN')
                     , access_token=os.getenv('TWEEPY_ACCESS_TOKEN')
                     , access_token_secret=os.getenv('TWEEPY_ACCESS_TOKEN_SECRET')
                     , wait_on_rate_limit = True)

In [4]:
class tweets:
    
    def __init__(self, username: str):
        self._username = username
        self._userId = client.get_user(username=self._username).data.data['id']

    def return_tweets_dataframe(self, nr_results = None):
        """
        Description:
            Return a pandas DataFrame wich contains tweets from a user.
            The DataFrame has the following schema:
                date: Date the tweet was posted in format %Y-%m-%d %H:%M:%S.
              , id_tweet: Unique identifier of tweet.
              , text: Tweet content.
              , likes: Total number of likes.
              , users_likes: A key-value pair containing id and username from users who liked the tweet.
        Args:
            nr_results = Number of tweets. The Tweepy API only supports 100 tweets per request.
        
        Authors:
            @marcuscardona
        
        Since 01-2023.
        """

        # Setar número de resultados
        if nr_results in range(0, 101, 1):
            max_results = (100 if nr_results is None else nr_results)
        else:
            max_results = 100
            print('O número de resultados foi setado para 100')
        # Retornar os tweets do usuário
        tweets = client.get_users_tweets(id = self._userId
                                       , max_results = max_results
                                       , tweet_fields = ['created_at'])

        # Listas vazias para as informações retornadas por loop
        id_tweet_list = []
        text_tweet_list = []
        likes_count_tweet_list = []
        likes_users_tweet_list  = []
        date_tweet_list = []

        for tweet in tweets.data:
            id_tweet = tweet.id
            text_tweet = tweet.text
            date_tweet = tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")
            #
            liked_tweet_object = client.get_liking_users(id = id_tweet)
            
            likes_count_tweet = liked_tweet_object.meta['result_count']
            # Retornar usuários que curtiram o tweet
            likes_users_tweet_dict = {}
            try:
                for i in liked_tweet_object.data:
                    id_user_like = i['id']
                    username_user_like = i['username']
                    likes_users_tweet_dict.update({id_user_like:username_user_like})
            except:
                pass
            
            # Apendar todas as informações
            date_tweet_list.append(date_tweet)
            id_tweet_list.append(id_tweet)
            text_tweet_list.append(text_tweet)
            likes_count_tweet_list.append(likes_count_tweet)
            likes_users_tweet_list.append(likes_users_tweet_dict)

        # Colunas para criar DF
        columns_to_stack = [
            date_tweet_list,id_tweet_list, text_tweet_list
          , likes_count_tweet_list,likes_users_tweet_list
            ]
        columns_names = [
            'date', 'id_tweet', 'text'
          , 'likes', 'users_likes'
        ]

        # Criar DataFrame
        df = pd.DataFrame(np.column_stack(columns_to_stack), columns = columns_names)
        df.insert(loc=1, column='username', value=self._username)

        return df

In [5]:
t = tweets(username='brisssma')
teste=t.return_tweets_dataframe(nr_results = 10)
teste

,date,username,id_tweet,text,likes,users_likes
0,2023-01-26 13:23:29,brisssma,1618600513221844992,@sofiacrvo @danicatzs doenca mental identifica...,1,{961703371718713344: 'danicatzs'}
1,2023-01-26 13:19:01,brisssma,1618599392428314626,@sofiacrvo @danicatzs sofia,1,{961703371718713344: 'danicatzs'}
2,2023-01-26 13:18:23,brisssma,1618599230335250433,@danicatzs heheheh,0,{}
3,2023-01-26 13:12:45,brisssma,1618597812220080131,@xenadamata literalmente impossivel!!!,0,{}
4,2023-01-12 15:17:24,brisssma,1613555752462962690,@tyceanne eu so vi essa tatu pensando tyce VIC...,0,{}
5,2023-01-03 10:03:31,brisssma,1610215272303976448,RT @juliaquaresmaaa: 🚨 QUADRINISTAS 🚨 tenho um...,0,{}
6,2022-12-19 14:24:07,brisssma,1604845034779873281,@kamnuto @tyceanne @jhonata____ TENHO CERTEZA ...,1,{1057425766391123970: 'kamnuto'}
7,2022-12-18 16:45:23,brisssma,1604518197079773186,@leom4asdasd porra leo 😩😩😩😩,0,{}
8,2022-12-18 16:32:54,brisssma,1604515054753308672,toda vez que o galvao fala griezman eu escuto ...,7,"{1582281016969920513: 'Jenna_9568', 2304255402..."
9,2022-12-15 22:25:59,brisssma,1603516747964973056,lote de corno @LATAM_BRA,2,"{961703371718713344: 'danicatzs', 337123799: '..."
